# Evaluate the output of the wayveslam_visual_odometry step
After switching from using the intrinsics database intrinsics to using the corrected calibration table intrinsics, we want to make sure that the vo trajectories are still reasonable. For this we simply compare the current vo trajectory on databricks with the ones that were generated using the corrected calibration table intrinsics.

In [9]:
from wayve.services.data.lakehouse.common.external_access.wayve_delta_table import WayveDeltaTable
import pandas as pd
from pathlib import Path
from typing import Dict, no_type_check
import pandas as pd
import numpy as np
import torch
from scipy.spatial.transform import Rotation as R
import json
from bokeh.io import output_file, save

from bokeh.palettes import Category10
from bokeh.plotting import figure

from wayve.core.ai.geometry.trajectory import Trajectory3D
from wayve.core.ai.geometry.transform_3d import Transform3D


from wayve.core.ai.geometry.frame_reference import SerializedFrameRef
from wayve.core.ai.geometry.rotation_3d import RotationVector3D
from wayve.core.ai.geometry.vector_3d import Vector3D
from wayve.core.data.schema.state.wayveslam import SensorPoseSchema

In [ ]:
VO_MAIN_CLOUD_TABLE = ""
VO_BRANCH_CLOUD_TABLE = ""

OUT_DIR = "/mnt/remote/data/benjin/tmp_share/notebooks/step_testing/visual_odometry_out"

In [10]:
def get_vo_df(table_path: str) -> pd.DataFrame:
    """
    Get VO poses from delta tables for run ID. Renames transform columns to WayveSLAM convention
    """
    df = WayveDeltaTable(path=table_path).to_pandas_dataframe()
    return df

def sensorpose_to_transform3d(sensorpose_df: pd.DataFrame) -> Transform3D:
    """
    Convert sensorpose dataframe to Transform3D
    """
    rotation = R.from_quat(
        np.stack(
            [
                sensorpose_df[SensorPoseSchema.quaternion_x.short_field_name],
                sensorpose_df[SensorPoseSchema.quaternion_y.short_field_name],
                sensorpose_df[SensorPoseSchema.quaternion_z.short_field_name],
                sensorpose_df[SensorPoseSchema.quaternion_w.short_field_name],
            ],
            axis=1,
        )
    )

    translation = Vector3D(
        torch.from_numpy(
            np.stack(
                [
                    sensorpose_df[SensorPoseSchema.x_position_m.short_field_name],
                    sensorpose_df[SensorPoseSchema.y_position_m.short_field_name],
                    sensorpose_df[SensorPoseSchema.z_position_m.short_field_name],
                ],
                axis=1,
            )
        )
    )

    batch_shape = translation.batch_shape
    source_frame = sensorpose_df[SensorPoseSchema.source_frame.short_field_name].to_numpy()
    destination_frame = sensorpose_df[SensorPoseSchema.destination_frame.short_field_name].to_numpy()

    return Transform3D(
        translation=translation,
        rotation=RotationVector3D(torch.from_numpy(rotation.as_rotvec())),
        from_frame=SerializedFrameRef.from_frame_ref(source_frame, batch_shape),
        to_frame=SerializedFrameRef.from_frame_ref(destination_frame, batch_shape),
    )

def sensorpose_to_trajectory3d(sensorpose_df: pd.DataFrame) -> Trajectory3D:
    """
    Convert sensorpose dataframe to Trajectory3D
    """
    transforms = sensorpose_to_transform3d(sensorpose_df)
    timestamps = sensorpose_df["timestamp_unixus"].to_numpy()
    return Trajectory3D(
        timestamps_us=torch.from_numpy(timestamps),
        transforms=transforms,
    )

@no_type_check
def plot_multiple_trajectories(plot_source: Dict[str, Trajectory3D], title: str) -> figure:
    """Plots a dict of trajectories with no legends"""
    bev_plot = figure(match_aspect=True)
    colors = Category10[10]

    for i, key in enumerate(plot_source):
        bev_plot.line(
            x=(plot_source[key].translation.forward_left_up).numpy().T[0],
            y=(plot_source[key].translation.forward_left_up).numpy().T[1],
            line_width=2,
            color=colors[i % len(colors)],
            legend_label=key,
        )

    bev_plot.title = title
    return bev_plot

def plot_trajectory_alignment(
    orig_poses: Trajectory3D,
    new_poses: Trajectory3D,
    output_path: Path,
):
    # PLOTTING
    fig = plot_multiple_trajectories(
        title='compare',
        plot_source={
            'vo_original': orig_poses,
            'vo_new': new_poses,
        }
    )
    output_file(output_path)
    save(fig)

def compare_trajectories(out_dir: str):
    out_dir = Path(out_dir)

    vo_main_df = get_vo_df(VO_MAIN_CLOUD_TABLE)
    vo_branch_df = get_vo_df(VO_BRANCH_CLOUD_TABLE)

    vo_main_trajectories = {
        run_id: sensorpose_to_trajectory3d(group)
        for run_id, group in vo_main_df.groupby('run_id')
    }

    vo_branch_trajectories = {
        run_id: sensorpose_to_trajectory3d(group)
        for run_id, group in vo_branch_df.groupby('run_id')
    }

    for run_id in vo_main_trajectories:
        vo_main_traj = vo_main_trajectories[run_id]
        vo_branch_traj = vo_branch_trajectories[run_id]

        plot_trajectory_alignment(vo_main_traj, vo_branch_traj, out_dir / f"vo_{run_id}.html")

compare_trajectories(OUT_DIR)